In [1]:
import sys

print("Используем Python {}".format(sys.version))

Используем Python 3.5.2 (default, Sep 14 2017, 22:51:06) 
[GCC 5.4.0 20160609]


In [2]:
import pandas as pd

print("Используем pandas {}".format(pd.__version__))

Используем pandas 0.21.0


In [3]:
import scipy

print("Используем scipy {}".format(scipy.__version__))

Используем scipy 1.0.0


In [6]:
# эта функция нам поможет считывать данные
from os import path

data_dir = "/home/ubuntu/data/movielens/ml-latest-small"
def read_csv(filename: str):
    data = pd.read_csv(path.join(data_dir, filename + ".csv"))
    return data

ratings = read_csv("ratings")
movies = read_csv("movies")

In [7]:
# пользователи ставят фильмам оценки
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [8]:
# про фильмы знаем названия и жанры
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [9]:
# максимальный ID фильма
last_movie_id = ratings["movieId"].max()
print(last_movie_id)
# максимальный ID пользователя
last_user_id = ratings["userId"].max()
print(last_user_id)

163949
671


In [12]:
# функция, которая красиво печатает информацию о разреженных матрицах
from scipy.sparse import csr_matrix

def sparse_info(sparse_matrix: csr_matrix):
    print("Размерности матрицы: {}".format(sparse_matrix.shape))
    print("Ненулевых элементов в матрице: {}".format(sparse_matrix.nnz))
    print("Доля ненулевых элементов: {}"
          .format(sparse_matrix.nnz / sparse_matrix.shape[0] / sparse_matrix.shape[1])
    )

In [13]:
user_x_item = ratings[["userId", "movieId"]].as_matrix()
user_x_item
user_item_matrix = csr_matrix(
    (
        [1] * len(user_x_item),
        (
            [pair[0] for pair in user_x_item],
            [pair[1] for pair in user_x_item],
        )
    ),
    shape=(last_user_id + 1, last_movie_id + 1)
)

sparse_info(user_item_matrix)

Размерности матрицы: (672, 163950)
Ненулевых элементов в матрице: 100004
Доля ненулевых элементов: 0.0009076881743853382


In [9]:
from scipy.spatial.distance import pdist, squareform

# pdist умеет всё "из коробки", но только с плотными матрицами
# плотная матрица не умещается в память - это нормально
similarity_matrix = squareform(pdist(user_item_matrix.todense().T))

MemoryError: 

In [14]:
# представление фильма в виде булевого вектора
movie_id = 260
print(user_item_matrix[:, movie_id])

  (4, 0)	1
  (7, 0)	1
  (8, 0)	1
  (15, 0)	1
  (17, 0)	1
  (18, 0)	1
  (19, 0)	1
  (20, 0)	1
  (21, 0)	1
  (22, 0)	1
  (23, 0)	1
  (25, 0)	1
  (26, 0)	1
  (30, 0)	1
  (31, 0)	1
  (34, 0)	1
  (36, 0)	1
  (40, 0)	1
  (41, 0)	1
  (42, 0)	1
  (44, 0)	1
  (55, 0)	1
  (56, 0)	1
  (57, 0)	1
  (62, 0)	1
  :	:
  (607, 0)	1
  (611, 0)	1
  (613, 0)	1
  (614, 0)	1
  (615, 0)	1
  (616, 0)	1
  (620, 0)	1
  (622, 0)	1
  (624, 0)	1
  (626, 0)	1
  (627, 0)	1
  (636, 0)	1
  (640, 0)	1
  (643, 0)	1
  (646, 0)	1
  (647, 0)	1
  (651, 0)	1
  (654, 0)	1
  (655, 0)	1
  (659, 0)	1
  (660, 0)	1
  (664, 0)	1
  (665, 0)	1
  (669, 0)	1
  (671, 0)	1


In [15]:
# теперь найдём матрицу схожести
similarity_matrix = user_item_matrix.T.dot(user_item_matrix)
sparse_info(similarity_matrix)

Размерности матрицы: (163950, 163950)
Ненулевых элементов в матрице: 21983224
Доля ненулевых элементов: 0.0008178403679890727


In [16]:
# top самых похожих фильмов
from scipy.sparse import find

similar_movies = (-similarity_matrix[movie_id]).todense().argsort().tolist()[0][1:6]

id_to_title_map = {
    pair[0]: pair[1] for pair in zip(movies["movieId"].tolist(), movies["title"].tolist())
}
print(id_to_title_map[movie_id])
[id_to_title_map[k] for k in similar_movies]

Star Wars: Episode IV - A New Hope (1977)


['Star Wars: Episode V - The Empire Strikes Back (1980)',
 'Star Wars: Episode VI - Return of the Jedi (1983)',
 'Forrest Gump (1994)',
 'Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981)',
 'Matrix, The (1999)']